# Inference Time of the Proposed Models

This notebook measures the time a model takes to generate an image. The time 
is selected as the quickest among 100 runs. It is not the average, as we
want to know how fast inference takes in an environment that is dedicated
to the task.

## Loads the Models

In [1]:
import tensorflow as tf

model_loaders = {
    'pix2pix': lambda: tf.keras.models.load_model('models/pix2pix/front-to-right', compile=False),
    # 'stargan': lambda: tf.keras.models.load_model(
    #     'models/selection/stargan-network-and-sampler/paired/target-multi/network-both', compile=False), # went to sbgames'24 originally
    'stargan': lambda: tf.keras.models.load_model('models/stargan', compile=False),
    'collagan': lambda: tf.keras.models.load_model('models/collagan', compile=False)
}

## Loads the Data

In [2]:
dataset_sizes = [912, 216, 294, 408, 12372]
        
def load_image(dataset, domain, index):
    path = f'datasets/{dataset}/test/{domain}/{index}.png'        
    return tf.cast(tf.io.decode_png(tf.io.read_file(path)), dtype=tf.float32) / 127.5 - 1.

sample_character = {
    'back': load_image('rpg-maker-xp', '0-back', 0),
    'left': load_image('rpg-maker-xp', '1-left', 0),
    'front': load_image('rpg-maker-xp', '2-front', 0),
    'right': load_image('rpg-maker-xp', '3-right', 0)
}


## Measures the Inference Time

In [3]:
import time

def measure_inference_time(model, runs=25000):
    best_time = float('inf')
    start_before_loop = time.time()
    for _ in range(runs):
        start_of_iteration = time.time()
        model()
        time_taken_this_iteration = time.time() - start_of_iteration
        
        if time_taken_this_iteration < best_time:
            best_time = time_taken_this_iteration
    time_taken_loop = time.time() - start_before_loop

    times = {"best time": best_time, "average time": time_taken_loop / runs }
    
    # clears keras session, to free memory
    tf.keras.backend.clear_session()
    
    return times    


# measures pix2pix time
def measure_pix2pix():
    model = model_loaders['pix2pix']()
    model_callback = lambda source_image: lambda: model(source_image[tf.newaxis, ...])
    
    # warmup the model
    model_callback(sample_character['front'])()
    
    stats = measure_inference_time(model_callback(sample_character['front']))
    del model
    return stats


# measures stargan-study-network-and-sampler time
def measure_stargan():
    model = model_loaders['stargan']()
    model_callback = lambda source_image, source_domain, target_domain: lambda: (
        model([source_image[tf.newaxis, ...], tf.constant([[source_domain]]), tf.constant([[target_domain]])]))
    
    # warmup the model
    model_callback(sample_character['front'], 2, 3)()

    stats = measure_inference_time(model_callback(sample_character['front'], 2, 3))
    del model
    return stats

# measures collagan time
def measure_collagan():
    model = model_loaders['collagan']()
    model_callback = lambda source_images, target_domain: lambda: model([tf.stack(source_images)[tf.newaxis, ...], tf.constant([[target_domain]])])
    
    # warmup the model
    model_callback([sample_character['back'], sample_character['left'], sample_character['front'], tf.zeros_like(sample_character['right'])], 3)()
    
    stats = measure_inference_time(model_callback([sample_character['back'], sample_character['left'], sample_character['front'], tf.zeros_like(sample_character['right'])], 3))
    del model
    return stats


In [4]:
collagan_time = measure_collagan()
stargan_time = measure_stargan()
pix2pix_time = measure_pix2pix()


D:\Projetos 2\dissertation-notebooks\.venv\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "
D:\Projetos 2\dissertation-notebooks\.venv\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


In [5]:
def print_results(results):
    header_format = "{:<8} | {:>17} | {:>17}"
    row_format = "{:<10} | {:>17.2f} | {:>17.2f}"
    print(header_format.format("Model", "Best Time (ms)", "Average Time (ms)"))
    for name, result in results.items():
        print(row_format.format(name, result['best time']*1000, result['average time']*1000))
        
        
print_results({
    'pix2pix': pix2pix_time,
    'stargan': stargan_time,
    'collagan': collagan_time
})

Model    |    Best Time (ms) | Average Time (ms)
pix2pix    |             30.92 |             33.53
stargan    |             46.03 |            144.58
collagan   |            110.03 |            158.09
